In [97]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [99]:
# importing Word2Vec
from gensim.models import Word2Vec
from gensim.models import FastText

In [100]:
X=pd.read_csv('/content/drive/My Drive/MinorProject2/BERT2/new_german.csv')
X.head()

,Unnamed: 0,tweet_id,text,task1,task2,ID
0,0,1133388798925189122,Deutsche rothaarige porno reife deutsche fraue...,NOT,NONE,hasoc_2020_de_2684
1,1,1131117000279961600,"Lehrstück auch, wie in der linken Jammerfemini...",HOF,NONE,hasoc_2020_de_2440
2,2,1127134592517980161,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NOT,NONE,hasoc_2020_de_1042
3,3,1128897106171842560,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NOT,NONE,hasoc_2020_de_774
4,4,1123576753199484928,@Junge_Freiheit Die Inkas hatten sich schon dä...,NOT,NONE,hasoc_2020_de_559


In [101]:
X=X[['text','task1']]
X

,text,task1
0,Deutsche rothaarige porno reife deutsche fraue...,NOT
1,"Lehrstück auch, wie in der linken Jammerfemini...",HOF
2,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NOT
3,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NOT
4,@Junge_Freiheit Die Inkas hatten sich schon dä...,NOT
...,...,...
3019,RT @alfunterwegs70: Sie belügen euch ganz öffe...,HOF
3020,"@SandraLustig13 Mannomann, dumme, verwöhnte, g...",HOF
3021,RT @helllud123: @ZDFheute @sven_giegold @Joerg...,HOF
3022,RT @den_tyske: ... gibt es wirklich noch Idiot...,HOF


In [102]:
df1=pd.read_csv('/content/drive/My Drive/MinorProject2/BERT2/german_test_1509.csv')
df1.head()

,tweet_id,text,task1,task2,ID
0,1129095874242650112,@derCarsti Boykottieren hört sich besser an. 💙💙,NOT,NONE,hasoc_2020_de_1053
1,1129004308396236800,"RT @ibikus31: Es wird spekuliert, ob Merkel ei...",NOT,NONE,hasoc_2020_de_671
2,1130896929355907080,Hat #Hitler wirklich den Krieg in der Wüste ve...,NOT,NONE,hasoc_2020_de_2977
3,1132251534329307136,RT @Beatrix_vStorch: #May tritt in UK unter Tr...,NOT,NONE,hasoc_2020_de_1746
4,1124941869115498496,@justmeDoro Eher nicht. Das Gänse hauen wieder...,NOT,NONE,hasoc_2020_de_2416


In [103]:
Y=df1[['text','task1']]
Y

,text,task1
0,@derCarsti Boykottieren hört sich besser an. 💙💙,NOT
1,"RT @ibikus31: Es wird spekuliert, ob Merkel ei...",NOT
2,Hat #Hitler wirklich den Krieg in der Wüste ve...,NOT
3,RT @Beatrix_vStorch: #May tritt in UK unter Tr...,NOT
4,@justmeDoro Eher nicht. Das Gänse hauen wieder...,NOT
...,...,...
521,RT @ChanMachtSo: SCHMERZEN!!!! Au!!! Mein Gehi...,NOT
522,Die Zerstörung der Grünen. https://t.co/SIYDJj...,NOT
523,"RT @PParzival: ""Antideutsche"" Pseudo-linke Ide...",NOT
524,Klug reden und dumm leben.,NOT


In [104]:
X.head()

,text,task1
0,Deutsche rothaarige porno reife deutsche fraue...,NOT
1,"Lehrstück auch, wie in der linken Jammerfemini...",HOF
2,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,NOT
3,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,NOT
4,@Junge_Freiheit Die Inkas hatten sich schon dä...,NOT


In [105]:
cols=['task1']
X[cols] = X[cols].apply(lambda x: pd.factorize(x)[0] + 0)
Y[cols] = Y[cols].apply(lambda x: pd.factorize(x)[0] + 0)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [106]:
X.head()

,text,task1
0,Deutsche rothaarige porno reife deutsche fraue...,0
1,"Lehrstück auch, wie in der linken Jammerfemini...",1
2,RT @NDRinfo: Die deutsche Klimaaktivistin Luis...,0
3,@ruhrbahn jeden Morgen eine neue „Fahrzeugstör...,0
4,@Junge_Freiheit Die Inkas hatten sich schon dä...,0


In [107]:
Y.head()

,text,task1
0,@derCarsti Boykottieren hört sich besser an. 💙💙,0
1,"RT @ibikus31: Es wird spekuliert, ob Merkel ei...",0
2,Hat #Hitler wirklich den Krieg in der Wüste ve...,0
3,RT @Beatrix_vStorch: #May tritt in UK unter Tr...,0
4,@justmeDoro Eher nicht. Das Gänse hauen wieder...,0


In [108]:
X_train=X['text']
X_train

0       Deutsche rothaarige porno reife deutsche fraue...
1       Lehrstück auch, wie in der linken Jammerfemini...
2       RT @NDRinfo: Die deutsche Klimaaktivistin Luis...
3       @ruhrbahn jeden Morgen eine neue „Fahrzeugstör...
4       @Junge_Freiheit Die Inkas hatten sich schon dä...
                              ...                        
3019    RT @alfunterwegs70: Sie belügen euch ganz öffe...
3020    @SandraLustig13 Mannomann, dumme, verwöhnte, g...
3021    RT @helllud123: @ZDFheute @sven_giegold @Joerg...
3022    RT @den_tyske: ... gibt es wirklich noch Idiot...
3023    RT @sueszling: Ich hasse Deutsche https://t.co...
Name: text, Length: 3024, dtype: object

In [109]:
y_train=X['task1']
y_train

0       0
1       1
2       0
3       0
4       0
       ..
3019    1
3020    1
3021    1
3022    1
3023    1
Name: task1, Length: 3024, dtype: int64

In [110]:
X_test=Y['text']
X_test

0        @derCarsti Boykottieren hört sich besser an. 💙💙
1      RT @ibikus31: Es wird spekuliert, ob Merkel ei...
2      Hat #Hitler wirklich den Krieg in der Wüste ve...
3      RT @Beatrix_vStorch: #May tritt in UK unter Tr...
4      @justmeDoro Eher nicht. Das Gänse hauen wieder...
                             ...                        
521    RT @ChanMachtSo: SCHMERZEN!!!! Au!!! Mein Gehi...
522    Die Zerstörung der Grünen. https://t.co/SIYDJj...
523    RT @PParzival: "Antideutsche" Pseudo-linke Ide...
524                           Klug reden und dumm leben.
525    Wissen wir schon lange....hat das die Merkel e...
Name: text, Length: 526, dtype: object

In [111]:
y_test=Y['task1']
y_test

0      0
1      0
2      0
3      0
4      0
      ..
521    0
522    0
523    0
524    0
525    0
Name: task1, Length: 526, dtype: int64

In [112]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3024,), (526,), (3024,), (526,))

In [113]:
def create_tokens(lines):
  #stop_words = set(nltk.corpus.stopwords.words('english'))
  t_line=lines.apply(lambda x: nltk.word_tokenize(x))
  #t_line=t_line.apply(lambda x: [w for w in x if not w in stop_words])
  return t_line

In [114]:
X_train=create_tokens(X_train)
print(X_train)
tokenizer= Tokenizer()
tokenizer.fit_on_texts(X_train)
tokenizer


0       [Deutsche, rothaarige, porno, reife, deutsche,...
1       [Lehrstück, auch, ,, wie, in, der, linken, Jam...
2       [RT, @, NDRinfo, :, Die, deutsche, Klimaaktivi...
3       [@, ruhrbahn, jeden, Morgen, eine, neue, „Fahr...
4       [@, Junge_Freiheit, Die, Inkas, hatten, sich, ...
                              ...                        
3019    [RT, @, alfunterwegs70, :, Sie, belügen, euch,...
3020    [@, SandraLustig13, Mannomann, ,, dumme, ,, ve...
3021    [RT, @, helllud123, :, @, ZDFheute, @, sven_gi...
3022    [RT, @, den_tyske, :, ..., gibt, es, wirklich,...
3023    [RT, @, sueszling, :, Ich, hasse, Deutsche, ht...
Name: text, Length: 3024, dtype: object


In [115]:
length=np.average(X_train.apply(lambda x:len(x)*len(x)))
length=np.sqrt(length)
print(length)
length=48

19.326695631203073


In [116]:
def encode_text(tokenizer, lines, length):
  encoded = tokenizer.texts_to_sequences(lines)
  padded = pad_sequences(encoded, maxlen=length, padding='post')
  return padded

In [117]:
word_index=tokenizer.word_index

In [118]:
X_test=create_tokens(X_test)
print(X_test)

0      [@, derCarsti, Boykottieren, hört, sich, besse...
1      [RT, @, ibikus31, :, Es, wird, spekuliert, ,, ...
2      [Hat, #, Hitler, wirklich, den, Krieg, in, der...
3      [RT, @, Beatrix_vStorch, :, #, May, tritt, in,...
4      [@, justmeDoro, Eher, nicht, ., Das, Gänse, ha...
                             ...                        
521    [RT, @, ChanMachtSo, :, SCHMERZEN, !, !, !, !,...
522    [Die, Zerstörung, der, Grünen, ., https, :, //...
523    [RT, @, PParzival, :, ``, Antideutsche, '', Ps...
524                   [Klug, reden, und, dumm, leben, .]
525    [Wissen, wir, schon, lange, ..., .hat, das, di...
Name: text, Length: 526, dtype: object


In [119]:
X_train=encode_text(tokenizer,X_train,length)
X_train
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[  22, 3067,   80, ...,    0,    0,    0],
       [3071,   47,    3, ...,    0,    0,    0],
       [   7,    1, 3075, ...,    0,    0,    0],
       ...,
       [   7,    1,  293, ...,    0,    0,    0],
       [   7,    1,  511, ...,    0,    0,    0],
       [   7,    1,  822, ...,    0,    0,    0]], dtype=int32)

In [120]:
X_test=encode_text(tokenizer,X_test,length)
X_test
#pd.DataFrame(train_x).to_csv("train_file.csv")

array([[   1,  128,   40, ...,    0,    0,    0],
       [   7,    1,    2, ...,    0,    0,    0],
       [  61,    8,  201, ...,    0,    0,    0],
       ...,
       [   7,    1, 3557, ...,    0,    0,    0],
       [ 861,   10,   46, ...,    0,    0,    0],
       [3549,   79,   34, ...,    0,    0,    0]], dtype=int32)

In [121]:
from keras.utils.np_utils import to_categorical
y_test = to_categorical(np.asarray(y_test))
y_train = to_categorical(np.asarray(y_train))
print(y_test)
print(y_train)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [122]:
# print(len(X_train))
# print(len(X_test))
# print(len(y_train))
# print(len(y_test))
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((3024, 48), (526, 48), (3024, 2), (526, 2))

In [123]:
# Creating embedding matrix
# Using 
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/MinorProject/crawl-300d-2M.vec.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [124]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/crawl-300d-2M.vec', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in crawl-300-2M.' % len(embeddings_index))

Total 1999996 word vectors in crawl-300-2M.


In [125]:
print((X_train))
print((X_test))
print((y_train))
print((y_test))

[[  22 3067   80 ...    0    0    0]
 [3071   47    3 ...    0    0    0]
 [   7    1 3075 ...    0    0    0]
 ...
 [   7    1  293 ...    0    0    0]
 [   7    1  511 ...    0    0    0]
 [   7    1  822 ...    0    0    0]]
[[   1  128   40 ...    0    0    0]
 [   7    1    2 ...    0    0    0]
 [  61    8  201 ...    0    0    0]
 ...
 [   7    1 3557 ...    0    0    0]
 [ 861   10   46 ...    0    0    0]
 [3549   79   34 ...    0    0    0]]
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [126]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
embedding_matrix.shape

(10671, 300)

In [127]:
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Input,LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
model=Sequential()
model.add(Embedding(input_dim = 10671, output_dim = 300, weights=[embedding_matrix], input_length=length,trainable=False))
model.add(LSTM(192, dropout=0.3, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 48, 300)           3201300   
_________________________________________________________________
lstm_2 (LSTM)                (None, 192)               378624    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 386       
Total params: 3,580,310
Trainable params: 379,010
Non-trainable params: 3,201,300
_________________________________________________________________


In [128]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=100,batch_size=256, verbose=2)

Epoch 1/100
12/12 - 2s - loss: 0.8295 - accuracy: 0.5351 - val_loss: 0.5676 - val_accuracy: 0.7452
Epoch 2/100
12/12 - 2s - loss: 0.6822 - accuracy: 0.5929 - val_loss: 0.6407 - val_accuracy: 0.7452
Epoch 3/100
12/12 - 2s - loss: 0.6739 - accuracy: 0.6121 - val_loss: 0.6067 - val_accuracy: 0.7452
Epoch 4/100
12/12 - 2s - loss: 0.6679 - accuracy: 0.6121 - val_loss: 0.6064 - val_accuracy: 0.7452
Epoch 5/100
12/12 - 2s - loss: 0.6692 - accuracy: 0.6121 - val_loss: 0.6273 - val_accuracy: 0.7452
Epoch 6/100
12/12 - 2s - loss: 0.6730 - accuracy: 0.6121 - val_loss: 0.5889 - val_accuracy: 0.7452
Epoch 7/100
12/12 - 2s - loss: 0.6690 - accuracy: 0.6121 - val_loss: 0.6234 - val_accuracy: 0.7452
Epoch 8/100
12/12 - 2s - loss: 0.6689 - accuracy: 0.6121 - val_loss: 0.6000 - val_accuracy: 0.7452
Epoch 9/100
12/12 - 2s - loss: 0.6701 - accuracy: 0.6121 - val_loss: 0.6408 - val_accuracy: 0.7452
Epoch 10/100
12/12 - 2s - loss: 0.6699 - accuracy: 0.6121 - val_loss: 0.5916 - val_accuracy: 0.7452
Epoch 11/

In [129]:
predictions_test = model.predict(X_test)

predictions_test

array([[9.9879348e-01, 1.2064683e-03],
       [9.9919790e-01, 8.0206787e-04],
       [9.9231839e-01, 7.6815966e-03],
       ...,
       [9.9862754e-01, 1.3724364e-03],
       [4.1812584e-01, 5.8187419e-01],
       [9.9898201e-01, 1.0179447e-03]], dtype=float32)

In [130]:
pred_test=np.zeros_like(predictions_test)
pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
# output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df1.index)
# output_DataFrame[0]=output_DataFrame[0].map({0:'NAG',1:'CAG',2:'OAG'})
# output_DataFrame.to_csv("Hindi Subtask B ou.csv")

In [131]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred_test))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87       392
           1       0.62      0.49      0.54       134

   micro avg       0.79      0.79      0.79       526
   macro avg       0.73      0.69      0.70       526
weighted avg       0.78      0.79      0.78       526
 samples avg       0.79      0.79      0.79       526



In [132]:
# df_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/hin_test.csv")
# df_test.head()

In [133]:
# model.save('HindiTaskB.h5')

In [134]:
# test_text=encode_text(tokenizer,df_test['Text'],length)
# test_text

In [135]:
# predictions_test = model.predict(test_text)

# predictions_test

In [136]:
# pred_test=np.zeros_like(predictions_test)
# pred_test[np.arange(len(predictions_test)),predictions_test.argmax(1)]=1
# output_DataFrame=pd.DataFrame(pred_test.argmax(axis=1),index=df_test['ID'])
# output_DataFrame[0]=output_DataFrame[0].map({0:'NAG',1:'CAG',2:'OAG'})
# output_DataFrame.to_csv("Hindi Subtask B CNN Fasttext output.csv")

In [137]:
  df3=df1[['tweet_id','ID']]
  df3

,tweet_id,ID
0,1129095874242650112,hasoc_2020_de_1053
1,1129004308396236800,hasoc_2020_de_671
2,1130896929355907080,hasoc_2020_de_2977
3,1132251534329307136,hasoc_2020_de_1746
4,1124941869115498496,hasoc_2020_de_2416
...,...,...
521,1124809878546128897,hasoc_2020_de_486
522,1132433240000798720,hasoc_2020_de_3388
523,1127366294255357958,hasoc_2020_de_2745
524,1124362090460975105,hasoc_2020_de_236


In [138]:
predictions_test1=predictions_test.argmax(1)
pred_test1=pd.Series(predictions_test1)
pred_test1

0      0
1      0
2      0
3      0
4      0
      ..
521    1
522    0
523    0
524    1
525    0
Length: 526, dtype: int64

In [139]:
df3['label']=pred_test1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [140]:
df3

,tweet_id,ID,label
0,1129095874242650112,hasoc_2020_de_1053,0
1,1129004308396236800,hasoc_2020_de_671,0
2,1130896929355907080,hasoc_2020_de_2977,0
3,1132251534329307136,hasoc_2020_de_1746,0
4,1124941869115498496,hasoc_2020_de_2416,0
...,...,...,...
521,1124809878546128897,hasoc_2020_de_486,1
522,1132433240000798720,hasoc_2020_de_3388,0
523,1127366294255357958,hasoc_2020_de_2745,0
524,1124362090460975105,hasoc_2020_de_236,1


In [141]:
df3['label'] = df3['label'].map({0: "NOT",1:"HOF"})
df3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet_id,ID,label
0,1129095874242650112,hasoc_2020_de_1053,NOT
1,1129004308396236800,hasoc_2020_de_671,NOT
2,1130896929355907080,hasoc_2020_de_2977,NOT
3,1132251534329307136,hasoc_2020_de_1746,NOT
4,1124941869115498496,hasoc_2020_de_2416,NOT
...,...,...,...
521,1124809878546128897,hasoc_2020_de_486,HOF
522,1132433240000798720,hasoc_2020_de_3388,NOT
523,1127366294255357958,hasoc_2020_de_2745,NOT
524,1124362090460975105,hasoc_2020_de_236,HOF


In [142]:
df3.to_csv('submission_DE_A.csv')

In [143]:
model.save("GermanDatasetSubTask1_withLSTM_Fasttext.h5")